In [2]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, Dropout, Dense, GRU, LSTM
from keras.optimizers import Adam
from keras.models import Sequential

In [58]:
# labels
with open('PEMS_trainlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    train_y = raw_line[0][1:-2].split(' ')

y_train = []
for label in train_y:
    y_train.append(int(label))
y_train = np.array(y_train) - 1
y_train = to_categorical(y_train, 7)

with open('PEMS_testlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    test_y = raw_line[0][1:-2].split(' ')

y_test = []
for label in test_y:
    y_test.append(int(label))
y_test = np.array(y_test) - 1
y_test = to_categorical(y_test, 7)

In [62]:
# data
with open('PEMS_train.txt', 'r', encoding='utf-8') as f:
    train_x_lines = f.readlines()

x_train = []
for line in train_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_train.append(values_list)

x_train = np.array(x_train)
x_train = x_train.reshape((267, 144, 963))
x_train = x_train.sum(axis=2)

x_train_diag = np.zeros((267, 144, 144))

for i, x in enumerate(x_train):
    x_train_diag[i] = np.diag(x)

with open('PEMS_test.txt', 'r', encoding='utf-8') as f:
    test_x_lines = f.readlines()

x_test = []
for line in test_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_test.append(values_list)

x_test = np.array(x_test)
x_test = x_test.reshape((173, 144, 963))
x_test = x_test.sum(axis=2)

x_test_diag = np.zeros((173, 144, 144))

for i, x in enumerate(x_test):
    x_test_diag[i] = np.diag(x)

In [66]:
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)
print()
print('x_train_diag.shape: ', x_train_diag.shape)
print('x_test_diag.shape: ', x_test_diag.shape)

x_train.shape:  (267, 144)
y_train.shape:  (267, 7)
x_test.shape:  (173, 144)
y_test.shape:  (173, 7)

x_train_diag.shape:  (267, 144, 144)
x_test_diag.shape:  (173, 144, 144)


In [79]:
model = Sequential()
model.add(LSTM(32, input_shape=x_train_diag[0].shape, return_sequences=False))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001))
model.fit(x_train_diag, y_train, batch_size=32, epochs=130, validation_data=(x_test_diag, y_test))

Epoch 1/130
9/9 [==============================] - 3s 98ms/step - loss: 1.9798 - val_loss: 1.9561
Epoch 2/130
9/9 [==============================] - 0s 34ms/step - loss: 1.9335 - val_loss: 1.9286
Epoch 3/130
9/9 [==============================] - 0s 34ms/step - loss: 1.8959 - val_loss: 1.8953
Epoch 4/130
9/9 [==============================] - 0s 34ms/step - loss: 1.8619 - val_loss: 1.8549
Epoch 5/130
9/9 [==============================] - 0s 34ms/step - loss: 1.8246 - val_loss: 1.8155
Epoch 6/130
9/9 [==============================] - 0s 34ms/step - loss: 1.7759 - val_loss: 1.7671
Epoch 7/130
9/9 [==============================] - 0s 34ms/step - loss: 1.7236 - val_loss: 1.7012
Epoch 8/130
9/9 [==============================] - 0s 34ms/step - loss: 1.6603 - val_loss: 1.6329
Epoch 9/130
9/9 [==============================] - 0s 34ms/step - loss: 1.5881 - val_loss: 1.5695
Epoch 10/130
9/9 [==============================] - 0s 35ms/step - loss: 1.5126 - val_loss: 1.4940
Epoch 11/130
9/9 [=

In [81]:
def model_accuracy(x, y, n=len(x_test_diag)):
    true_cnt = 0
    for _ in range(n):
        rand_indx = np.random.randint(len(x_test_diag))
        pred = np.argmax(model.predict(x_test_diag[rand_indx:rand_indx + 1], verbose=False), axis=1)[0]
        true = np.argmax(y_test[rand_indx])
        if pred == true:
            true_cnt += 1
    print('\nModel accuracy =', true_cnt / n * 100, '%')


model_accuracy(x_test_diag, y_test)


Model accuracy = 80.34682080924856 %
